In [497]:
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import matplotlib.pyplot as plt
import hvplot.pandas
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim
from os.path import exists
from urllib.request import urlretrieve
import holoviews as hv
from fiona.crs import from_epsg
from geopy.geocoders import Nominatim
from haversine import haversine, Unit
import leafmap

import warnings
warnings.filterwarnings('ignore')
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=300, frame_height=500))
#hvplot_defaults = {'tiles':None, 'cmap':'Viridis', 'colorbar':True}





In [502]:
pd.set_option('display.max_colwidth', None) # To avoid truncated display of DataFrames 

In [503]:
df = pd.read_excel("positiondump utc .xlsx", index_col = [0])


In [500]:
df

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
647751005,2023-06-14 18:15:03,0,52.203256,5.972714
647742914,2023-06-14 17:17:28,0,52.203289,5.972705
647742890,2023-06-14 17:17:23,0,52.203289,5.972705
647742726,2023-06-14 17:16:23,1,52.203792,5.975125
647742547,2023-06-14 17:15:23,1,52.200175,5.979859
...,...,...,...,...
631418653,2023-04-01 09:22:53,1,53.391285,5.282144
631418504,2023-04-01 09:21:53,1,53.389952,5.274539
631418385,2023-04-01 09:20:53,1,53.389564,5.270365


In [504]:
df.sort_values("DateTimeOfPosition")[:20]

,GPSpoint,DateTimeOfPosition,IgnitionOn
UnitpositionID,,,
631414692,"N53.389323,E5.270153",2023-04-01 08:50:30,0
631418242,"N53.389274,E5.269988",2023-04-01 09:19:54,1
631418385,"N53.389564,E5.270365",2023-04-01 09:20:53,1
631418504,"N53.389952,E5.274539",2023-04-01 09:21:53,1
631418653,"N53.391285,E5.282144",2023-04-01 09:22:53,1
631418767,"N53.391549,E5.285710",2023-04-01 09:23:53,1
631418902,"N53.392134,E5.283950",2023-04-01 09:24:53,1
631419027,"N53.391772,E5.284025",2023-04-01 09:25:53,1
631419170,"N53.392237,E5.284695",2023-04-01 09:26:53,0


In [505]:
df.sort_values("DateTimeOfPosition")[175:185]

,GPSpoint,DateTimeOfPosition,IgnitionOn
UnitpositionID,,,
631782245,"N53.357367,E5.217147",2023-04-03 15:10:33,1
631782679,"N53.357366,E5.217149",2023-04-03 15:11:33,1
631783060,"N53.357366,E5.217147",2023-04-03 15:12:33,1
631783482,"N53.357366,E5.217147",2023-04-03 15:13:33,1
631783927,"N53.356975,E5.217310",2023-04-03 15:14:33,1
631784354,"N53.356782,E5.217236",2023-04-03 15:15:33,0
631784359,"N53.356782,E5.217236",2023-04-03 15:15:34,0
631817093,"N53.357044,E5.217039",2023-04-03 17:14:40,1
631817254,"N53.177227,E5.412910",2023-04-03 17:15:39,1


In [454]:
dfcpy.iloc[181:185]  #        53.17723, 5.41291

,DateTimeOfPosition,geometry,direction,HaversineDistance,DurationDiff
181,2023-04-03 15:15:34,POINT (5.21724 53.35678),0.000000,0.000000,1.0
182,2023-04-03 17:14:40,POINT (5.21704 53.35704),335.831429,36.353382,7146.0
183,2023-04-03 17:15:39,POINT (5.41291 53.17723),146.837965,29508.025066,59.0
184,2023-04-03 17:16:39,POINT (5.41325 53.17733),63.659691,39.073642,60.0


In [506]:
df["long"] = df["GPSpoint"].apply(lambda x: x.split(",")[0])

In [507]:
df["lat"] = df["GPSpoint"].apply(lambda x: x.split(",")[-1])

In [508]:
#Convert Longs and Lats to floats. 

def cnvrtCoord(val: str) -> float:
    sgn = +1
    if val[0] == 'S' or val[0] == 'W':
        sgn = -1
    return float(val[1:]) * sgn  

df['long'] = [cnvrtCoord(x) for x in df['long'].tolist()]
df['lat'] = [cnvrtCoord(x) for x in df['lat'].tolist()]  

df.drop("GPSpoint", axis=1, inplace=True)

In [509]:
df.head()

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
647751005,2023-06-14 18:15:03,0,52.203256,5.972714
647742914,2023-06-14 17:17:28,0,52.203289,5.972705
647742890,2023-06-14 17:17:23,0,52.203289,5.972705
647742726,2023-06-14 17:16:23,1,52.203792,5.975125
647742547,2023-06-14 17:15:23,1,52.200175,5.979859


In [510]:
df['DateTimeOfPosition'] = pd.to_datetime(df['DateTimeOfPosition'])


# Trajectory

In [511]:
traj = mpd.Trajectory(df[["DateTimeOfPosition", "long", "lat"]], "Allsetra", x='lat', y='long', t='DateTimeOfPosition', crs=4326)


In [512]:
traj.df

,geometry
DateTimeOfPosition,
2023-04-01 08:50:30,POINT (5.27015 53.38932)
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
...,...
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)
2023-06-14 17:17:23,POINT (5.97271 52.20329)


In [513]:
traj.hvplot(line_width=8, tiles='EsriStreet') #EsriImagery


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Path.I   :Path   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (triangle_angle)

In [49]:
#traj.df["Address"] = traj.df.geometry.apply(lambda x: get_address_from_GeoPoint(x))

In [44]:
traj.df

,geometry
DateTimeOfPosition,
2023-04-01 08:50:30,POINT (5.27015 53.38932)
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
...,...
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)
2023-06-14 17:17:23,POINT (5.97271 52.20329)


# Stops

In [514]:
from datetime import timedelta

In [515]:
detector = mpd.TrajectoryStopDetector(traj)
stops = detector.get_stop_points(min_duration=timedelta(seconds=200), max_diameter=200)


In [516]:
stops

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-04-01 08:50:30,POINT (5.27015 53.38932),2023-04-01 08:50:30,2023-04-01 09:20:53,Allsetra,1823.0
Allsetra_2023-04-01 09:23:53,POINT (5.28454 53.39195),2023-04-01 09:23:53,2023-04-01 11:07:38,Allsetra,6225.0
Allsetra_2023-04-01 11:11:32,POINT (5.27045 53.38903),2023-04-01 11:11:32,2023-04-01 12:19:03,Allsetra,4051.0
Allsetra_2023-04-01 12:26:22,POINT (5.30668 53.38933),2023-04-01 12:26:22,2023-04-01 12:37:27,Allsetra,665.0
Allsetra_2023-04-01 12:44:19,POINT (5.26995 53.38891),2023-04-01 12:44:19,2023-04-01 14:47:48,Allsetra,7409.0
...,...,...,...,...,...
Allsetra_2023-06-13 12:49:16,POINT (4.91754 51.74379),2023-06-13 12:49:16,2023-06-13 13:19:15,Allsetra,1799.0
Allsetra_2023-06-13 13:49:15,POINT (5.06728 51.58905),2023-06-13 13:49:15,2023-06-13 14:48:58,Allsetra,3583.0
Allsetra_2023-06-13 16:16:57,POINT (5.97345 52.20354),2023-06-13 16:16:57,2023-06-14 06:43:06,Allsetra,51969.0


# Stop-Trajectory Splitter 

In [517]:
traj_col = mpd.StopSplitter(traj).split(max_diameter=200, min_duration=timedelta(seconds=180))


In [518]:
traj_col

TrajectoryCollection with 347 trajectories

In [46]:
traj_col.hvplot(line_width = 10)

In [65]:
stops.head(3)

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
Allsetra_2023-04-01 08:50:30,POINT (5.27015 53.38932),2023-04-01 08:50:30,2023-04-01 09:20:53,Allsetra,1823.0
Allsetra_2023-04-01 09:23:53,POINT (5.28454 53.39195),2023-04-01 09:23:53,2023-04-01 11:07:38,Allsetra,6225.0
Allsetra_2023-04-01 11:11:32,POINT (5.27045 53.38903),2023-04-01 11:11:32,2023-04-01 12:19:03,Allsetra,4051.0


In [63]:
traj_col.trajectories[0].df

,geometry
DateTimeOfPosition,
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)


In [60]:
traj_col.trajectories[1].df

,geometry
DateTimeOfPosition,
2023-04-01 11:07:38,POINT (5.28440 53.39160)
2023-04-01 11:08:38,POINT (5.27646 53.39017)
2023-04-01 11:09:38,POINT (5.27132 53.38898)
2023-04-01 11:10:38,POINT (5.26997 53.38895)
2023-04-01 11:11:32,POINT (5.26993 53.38894)


In [66]:
traj_col.trajectories[2].df

,geometry
DateTimeOfPosition,
2023-04-01 12:19:03,POINT (5.27096 53.38911)
2023-04-01 12:20:02,POINT (5.27744 53.39033)
2023-04-01 12:21:02,POINT (5.28470 53.39087)
2023-04-01 12:22:02,POINT (5.28488 53.38499)
2023-04-01 12:23:03,POINT (5.29203 53.38464)
2023-04-01 12:24:02,POINT (5.29822 53.38652)
2023-04-01 12:25:03,POINT (5.30702 53.38917)
2023-04-01 12:26:03,POINT (5.30687 53.38941)
2023-04-01 12:26:22,POINT (5.30668 53.38932)


# Finalizing 

In [153]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(traj_col.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [154]:
new_df = pd.DataFrame(data)

In [155]:
new_df

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,2023-04-01 09:20:53,2023-04-01 09:23:53,"(5.270365, 53.389564)","(5.28571, 53.391549)",1.720386,0 days 00:03:00
1,2023-04-01 11:07:38,2023-04-01 11:11:32,"(5.284403, 53.391597)","(5.269931, 53.388944)",1.635806,0 days 00:03:54
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19
3,2023-04-01 12:37:27,2023-04-01 12:44:19,"(5.306676, 53.389332)","(5.269951, 53.388914)",4.083902,0 days 00:06:52
4,2023-04-01 14:47:48,2023-04-01 14:52:01,"(5.272201, 53.389177)","(5.28636, 53.402797)",2.180136,0 days 00:04:13
...,...,...,...,...,...,...
334,2023-06-13 12:26:03,2023-06-13 12:49:16,"(4.846854, 51.931568)","(4.917714, 51.74426)",22.197637,0 days 00:23:13
335,2023-06-13 13:19:15,2023-06-13 13:49:15,"(4.916819, 51.742863)","(5.0676, 51.589485)",23.869822,0 days 00:30:00
336,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59
337,2023-06-14 06:43:06,2023-06-14 07:38:21,"(5.974457, 52.203751)","(5.376879, 52.154919)",66.667055,0 days 00:55:15


In [150]:
new_df[["trip_distance_in_km", "trip_duration"]].describe()

,trip_distance_in_km
count,339.000000
mean,33.758662
std,36.858872
min,0.032817
25%,2.052191
50%,19.001563
75%,66.467911
max,197.137965


In [250]:
new_df.sort_values("trip_duration")[100:]

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
258,2023-05-27 22:50:11,2023-05-27 22:58:11,"(5.97434, 52.203637)","(6.007143, 52.208205)",3.682347,0 days 00:08:00
279,2023-06-01 08:46:31,2023-06-01 08:54:31,"(5.972844, 52.204151)","(5.996012, 52.212297)",2.729137,0 days 00:08:00
299,2023-06-06 10:28:12,2023-06-06 10:36:30,"(6.598152, 53.20255)","(6.612842, 53.219873)",2.515840,0 days 00:08:18
118,2023-04-29 13:04:25,2023-04-29 13:12:49,"(6.097459, 52.507347)","(6.106652, 52.505466)",1.043158,0 days 00:08:24
110,2023-04-28 08:39:23,2023-04-28 08:48:22,"(5.366711, 52.188312)","(5.376832, 52.154925)",3.863700,0 days 00:08:59
...,...,...,...,...,...,...
157,2023-05-09 07:27:54,2023-05-09 09:12:37,"(5.977874, 52.193452)","(4.933096, 52.307976)",116.863749,0 days 01:44:43
186,2023-05-16 07:25:00,2023-05-16 09:18:01,"(5.974655, 52.203694)","(5.06647, 51.586308)",121.931356,0 days 01:53:01
270,2023-05-31 06:20:13,2023-05-31 08:13:31,"(5.975679, 52.204116)","(4.236802, 51.857055)",197.137965,0 days 01:53:18
167,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58


In [229]:
traj_col.trajectories[0].df

datetime.time(9, 20, 53)

In [264]:
maxx = 0
for trajj in traj_col.trajectories:
    #maxx = max((trajj.df.index[-1] - trajj.df.index[0]).seconds, maxx)
    if 18652 == ((trajj.df.index[-1] - trajj.df.index[0]).seconds):
        debug = trajj

In [266]:
debug.df

,geometry,direction
DateTimeOfPosition,,
2023-05-22 11:27:05,POINT (5.56770 52.04229),0.000000
2023-05-22 11:28:05,POINT (5.56458 52.04350),302.283639
2023-05-22 11:29:05,POINT (5.55641 52.04402),275.942943
2023-05-22 11:30:05,POINT (5.53484 52.04303),265.718831
2023-05-22 11:31:05,POINT (5.50756 52.04673),282.454679
2023-05-22 11:32:05,POINT (5.47985 52.04927),278.486159
2023-05-22 11:33:05,POINT (5.45228 52.05114),276.297660
2023-05-22 11:34:05,POINT (5.42638 52.05222),273.907017
2023-05-22 11:35:05,POINT (5.40147 52.05522),281.071288


In [328]:
df[3850:3900] #      (52.154662, 5.374686)   52.158749, 5.379752

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
642275106,2023-05-22 17:33:03,1,52.159051,5.377499
642275082,2023-05-22 17:33:03,1,52.159051,5.377499
642274930,2023-05-22 17:32:03,1,52.159051,5.377499
642274750,2023-05-22 17:31:04,1,52.159051,5.377499
642253160,2023-05-22 16:40:28,0,52.159051,5.377499
642253048,2023-05-22 16:39:57,1,52.159051,5.377499
642252804,2023-05-22 16:38:57,1,52.159051,5.377499
642252588,2023-05-22 16:37:57,1,52.158749,5.379752
642166980,2023-05-22 12:06:50,0,52.154872,5.376606


In [329]:
traj.df

,geometry,direction
DateTimeOfPosition,,
2023-04-01 08:50:30,POINT (5.27015 53.38932),243.528683
2023-04-01 09:19:54,POINT (5.26999 53.38927),243.528683
2023-04-01 09:20:53,POINT (5.27036 53.38956),37.785736
2023-04-01 09:21:53,POINT (5.27454 53.38995),81.138877
2023-04-01 09:22:53,POINT (5.28214 53.39129),73.617975
...,...,...
2023-06-14 17:15:23,POINT (5.97986 52.20018),6.458632
2023-06-14 17:16:23,POINT (5.97513 52.20379),321.267067
2023-06-14 17:17:23,POINT (5.97271 52.20329),251.266558


In [493]:
dfcpy = traj.df.copy()

In [464]:
dfcpy = new_df.copy()

In [444]:
dfcpy = dfcpy.reset_index()

In [475]:
dfcpy

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
0,2023-04-01 09:20:53,2023-04-01 09:23:53,"(5.270365, 53.389564)","(5.28571, 53.391549)",1.720386,0 days 00:03:00
1,2023-04-01 11:07:38,2023-04-01 11:11:32,"(5.284403, 53.391597)","(5.269931, 53.388944)",1.635806,0 days 00:03:54
2,2023-04-01 12:19:03,2023-04-01 12:26:22,"(5.270962, 53.389109)","(5.306682, 53.389323)",3.971959,0 days 00:07:19
3,2023-04-01 12:37:27,2023-04-01 12:44:19,"(5.306676, 53.389332)","(5.269951, 53.388914)",4.083902,0 days 00:06:52
4,2023-04-01 14:47:48,2023-04-01 14:52:01,"(5.272201, 53.389177)","(5.28636, 53.402797)",2.180136,0 days 00:04:13
...,...,...,...,...,...,...
334,2023-06-13 12:26:03,2023-06-13 12:49:16,"(4.846854, 51.931568)","(4.917714, 51.74426)",22.197637,0 days 00:23:13
335,2023-06-13 13:19:15,2023-06-13 13:49:15,"(4.916819, 51.742863)","(5.0676, 51.589485)",23.869822,0 days 00:30:00
336,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59
337,2023-06-14 06:43:06,2023-06-14 07:38:21,"(5.974457, 52.203751)","(5.376879, 52.154919)",66.667055,0 days 00:55:15


In [494]:


# Iterate through the DataFrame rows
for i in range(1, len(dfcpy)):
    # Get the coordinates of the current and previous points
    lon1, lat1 = dfcpy["geometry"].iloc[i].x, dfcpy["geometry"].iloc[i].y
    lon2, lat2 = dfcpy["geometry"].iloc[i-1].x, dfcpy["geometry"].iloc[i-1].y
    
    # Calculate the Haversine distance between the points
    dist = haversine((lon1, lat1), (lon2, lat2), unit=Unit.METERS)
    
    # Update the Haversine distance in the DataFrame
    dfcpy.at[i, 'HaversineDistance'] = dist

In [495]:
dfcpy

,geometry,direction,HaversineDistance
DateTimeOfPosition,,,
2023-04-01 08:50:30,POINT (5.27015 53.38932),243.528683,NaN
2023-04-01 09:19:54,POINT (5.26999 53.38927),243.528683,NaN
2023-04-01 09:20:53,POINT (5.27036 53.38956),37.785736,NaN
2023-04-01 09:21:53,POINT (5.27454 53.38995),81.138877,NaN
2023-04-01 09:22:53,POINT (5.28214 53.39129),73.617975,NaN
...,...,...,...
10622,None,NaN,783.767960
10623,None,NaN,661.134636
10624,None,NaN,274.781661


In [492]:
#dfcpy['DurationDiff'] = None

# Iterate through the DataFrame rows
for i in range(1, len(dfcpy)):
    start_time = dfcpy["DateTimeOfPosition"][i-1]
    end_time = dfcpy["DateTimeOfPosition"][i]
    diff = end_time - start_time
    dfcpy.at[i, 'DurationDiff'] = diff.seconds

KeyError: 'DateTimeOfPosition'

In [449]:
dfcpy

,DateTimeOfPosition,geometry,direction,HaversineDistance,DurationDiff
0,2023-04-01 08:50:30,POINT (5.27015 53.38932),243.528683,NaN,NaN
1,2023-04-01 09:19:54,POINT (5.26999 53.38927),243.528683,19.132581,1764.0
2,2023-04-01 09:20:53,POINT (5.27036 53.38956),37.785736,52.805309,59.0
3,2023-04-01 09:21:53,POINT (5.27454 53.38995),81.138877,466.112333,60.0
4,2023-04-01 09:22:53,POINT (5.28214 53.39129),73.617975,858.422248,60.0
...,...,...,...,...,...
10622,2023-06-14 17:15:23,POINT (5.97986 52.20018),6.458632,783.767960,60.0
10623,2023-06-14 17:16:23,POINT (5.97513 52.20379),321.267067,661.134636,60.0
10624,2023-06-14 17:17:23,POINT (5.97271 52.20329),251.266558,274.781661,60.0
10625,2023-06-14 17:17:28,POINT (5.97271 52.20329),0.000000,0.000000,5.0


In [458]:
dfcpy = dfcpy.drop(index=183, axis=0)

In [463]:
dfcpy[dfcpy["HaversineDistance"] > 6000]

,DateTimeOfPosition,geometry,direction,HaversineDistance,DurationDiff
4922,2023-05-09 13:32:37,POINT (5.00558 52.32446),53.775205,6058.170267,60.0
6553,2023-05-19 06:38:12,POINT (5.86848 52.18702),280.351753,6746.796254,120.0
6558,2023-05-19 06:44:12,POINT (5.70159 52.19178),261.188099,6453.656784,120.0
8421,2023-05-31 15:42:56,POINT (5.00602 52.32490),53.903782,6166.816066,60.0


In [490]:

plt = sns.scatterplot(x=dfcpy.DurationDiff, y=dfcpy.HaversineDistance)

AttributeError: 'DataFrame' object has no attribute 'DurationDiff'

In [451]:
import numpy as np
np.argmax(dfcpy.HaversineDistance[1:].values)

182

In [452]:
dfcpy.iloc[181:185]

,DateTimeOfPosition,geometry,direction,HaversineDistance,DurationDiff
181,2023-04-03 15:15:34,POINT (5.21724 53.35678),0.000000,0.000000,1.0
182,2023-04-03 17:14:40,POINT (5.21704 53.35704),335.831429,36.353382,7146.0
183,2023-04-03 17:15:39,POINT (5.41291 53.17723),146.837965,29508.025066,59.0
184,2023-04-03 17:16:39,POINT (5.41325 53.17733),63.659691,39.073642,60.0


In [532]:
traj.df

,geometry
DateTimeOfPosition,
2023-04-01 08:50:30,POINT (5.27015 53.38932)
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
...,...
2023-06-14 17:15:23,POINT (5.97986 52.20018)
2023-06-14 17:16:23,POINT (5.97513 52.20379)
2023-06-14 17:17:23,POINT (5.97271 52.20329)


In [536]:
df[10680:]

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
631444538,2023-04-01 12:20:02,1,53.390326,5.277440
631444403,2023-04-01 12:19:03,1,53.389109,5.270962
631433754,2023-04-01 11:11:32,0,53.388944,5.269931
631433630,2023-04-01 11:10:38,1,53.388952,5.269975
631433458,2023-04-01 11:09:38,1,53.388979,5.271317
631433307,2023-04-01 11:08:38,1,53.390167,5.276463
631433148,2023-04-01 11:07:38,1,53.391597,5.284403
631432987,2023-04-01 11:06:38,1,53.392189,5.284687
631419170,2023-04-01 09:26:53,0,53.392237,5.284695


In [530]:
splitted_traj.trajectories[0].df

,geometry
DateTimeOfPosition,
2023-04-01 09:19:54,POINT (5.26999 53.38927)
2023-04-01 09:20:53,POINT (5.27036 53.38956)
2023-04-01 09:21:53,POINT (5.27454 53.38995)
2023-04-01 09:22:53,POINT (5.28214 53.39129)
2023-04-01 09:23:53,POINT (5.28571 53.39155)
2023-04-01 09:24:53,POINT (5.28395 53.39213)
2023-04-01 09:25:53,POINT (5.28402 53.39177)
2023-04-01 09:26:53,POINT (5.28470 53.39224)


In [531]:
splitted_traj.trajectories[1].df

,geometry
DateTimeOfPosition,
2023-04-01 11:06:38,POINT (5.28469 53.39219)
2023-04-01 11:07:38,POINT (5.28440 53.39160)
2023-04-01 11:08:38,POINT (5.27646 53.39017)
2023-04-01 11:09:38,POINT (5.27132 53.38898)
2023-04-01 11:10:38,POINT (5.26997 53.38895)
2023-04-01 11:11:32,POINT (5.26993 53.38894)


In [613]:
combined_dfs.sort_index()[2830:2850]

,geometry
DateTimeOfPosition,
2023-04-24 07:18:35,POINT (5.39176 52.15303)
2023-04-24 07:19:35,POINT (5.38465 52.15349)
2023-04-24 07:20:35,POINT (5.38384 52.15587)
2023-04-24 07:21:35,POINT (5.37969 52.15772)
2023-04-24 07:23:33,POINT (5.37680 52.15495)
2023-04-24 13:39:42,POINT (5.37671 52.15536)
2023-04-24 13:39:43,POINT (5.37671 52.15536)
2023-04-24 16:47:02,POINT (5.37693 52.15523)
2023-04-24 16:48:00,POINT (5.38072 52.15870)


In [612]:
df[7780:7800]

,DateTimeOfPosition,IgnitionOn,long,lat
UnitpositionID,,,,
636279819,2023-04-24 16:50:00,1,52.158941,5.377339
636279664,2023-04-24 16:49:00,1,52.158799,5.377356
636279519,2023-04-24 16:48:00,1,52.158697,5.380719
636279339,2023-04-24 16:47:02,1,52.155234,5.376927
636278271,2023-04-24 16:43:04,0,52.155217,5.377147
636224062,2023-04-24 13:39:43,0,52.155357,5.376705
636224170,2023-04-24 13:39:42,0,52.155357,5.376705
636195161,2023-04-24 13:19:29,0,52.155357,5.376705
636132427,2023-04-24 07:23:33,0,52.154952,5.376802


In [617]:
haversine((52.158697, 5.380719), (52.155234, 5.376927))

0.46389109689387

In [544]:
data = {}
start_addrs = []
end_addrs = []
start_timestamps = []
end_timestamps = []
start_longs_and_lats = []
end_longs_and_lats = []
distances = []
durations = []

for idx, trajectory in enumerate(splitted_traj.trajectories):
        
    start_time = trajectory.df.index[0]
    end_time = trajectory.df.index[-1]
    
    start_timestamps.append(start_time)
    end_timestamps.append(end_time)
    
    #durations.append(str(end_time - start_time).split("days")[-1].split(".")[0])
    durations.append(end_time - start_time)

    start_xy = (trajectory.df["geometry"].x[0], trajectory.df["geometry"].y[0])
    end_xy = (trajectory.df["geometry"].x[-1], trajectory.df["geometry"].y[-1])
    
    
#     start_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[0]))
#     end_addrs.append(get_address_from_GeoPoint(trajectory.df["geometry"].iloc[-1]))

    start_longs_and_lats.append(start_xy)
    end_longs_and_lats.append(end_xy)
    
    distances.append(haversine(start_xy, end_xy, unit=Unit.KILOMETERS))
    
# data["start_trip_addr"] = start_addrs
# data["end_trip_addr"] = end_addrs

data["start_trip_time"] = start_timestamps
data["end_trip_time"] = end_timestamps

data["start_gps_point"] = start_longs_and_lats
data["end_gps_point"] = end_longs_and_lats

data["trip_distance_in_km"] = distances

data["trip_duration"] = durations



In [546]:
splitted_df = pd.DataFrame(data)

In [557]:
splitted_df[85:]

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
85,2023-04-24 13:39:42,2023-04-24 13:39:43,"(5.376705, 52.155357)","(5.376705, 52.155357)",0.000000,0 days 00:00:01
86,2023-04-24 16:47:02,2023-04-24 16:50:29,"(5.376927, 52.155234)","(5.377339, 52.158941)",0.412935,0 days 00:03:27
87,2023-04-24 17:42:26,2023-04-24 18:28:36,"(5.377339, 52.158941)","(5.974082, 52.203512)",66.537907,0 days 00:46:10
88,2023-04-25 06:35:44,2023-04-25 07:32:07,"(5.974139, 52.203824)","(5.600667, 51.746678)",65.438853,0 days 00:56:23
89,2023-04-25 07:35:41,2023-04-25 08:08:13,"(5.600678, 51.746667)","(5.149545, 51.68995)",50.555202,0 days 00:32:32
...,...,...,...,...,...,...
320,2023-06-13 12:53:24,2023-06-13 12:53:52,"(4.917784, 51.744315)","(4.917539, 51.74378)",0.065231,0 days 00:00:28
321,2023-06-13 13:17:16,2023-06-13 13:52:28,"(4.917536, 51.743775)","(5.067226, 51.588894)",23.903979,0 days 00:35:12
322,2023-06-13 14:48:58,2023-06-13 16:16:57,"(5.067325, 51.58887)","(5.973399, 52.203352)",121.556916,0 days 01:27:59
323,2023-06-14 06:42:08,2023-06-14 07:38:21,"(5.973082, 52.203594)","(5.376879, 52.154919)",66.513258,0 days 00:56:13


In [548]:
splitted_df["trip_duration"].describe()

count                          325
mean     0 days 00:30:45.990769230
std      0 days 00:26:01.865196452
min                0 days 00:00:01
25%                0 days 00:07:29
50%                0 days 00:23:02
75%                0 days 00:47:19
max                0 days 01:55:20
Name: trip_duration, dtype: object

In [549]:
splitted_df["trip_distance_in_km"].describe()

count    325.000000
mean      35.133931
std       37.960051
min        0.000000
25%        2.031030
50%       19.112523
75%       66.521880
max      196.985192
Name: trip_distance_in_km, dtype: float64

In [550]:
splitted_df.sort_values("trip_distance_in_km")

,start_trip_time,end_trip_time,start_gps_point,end_gps_point,trip_distance_in_km,trip_duration
63,2023-04-17 17:41:45,2023-04-17 17:41:58,"(5.377497, 52.158842)","(5.377497, 52.158842)",0.000000,0 days 00:00:13
183,2023-05-15 07:53:14,2023-05-15 07:53:18,"(5.376806, 52.154914)","(5.376806, 52.154914)",0.000000,0 days 00:00:04
85,2023-04-24 13:39:42,2023-04-24 13:39:43,"(5.376705, 52.155357)","(5.376705, 52.155357)",0.000000,0 days 00:00:01
314,2023-06-12 09:48:38,2023-06-12 09:49:01,"(5.376883, 52.154699)","(5.376879, 52.154716)",0.001934,0 days 00:00:23
294,2023-06-06 17:46:55,2023-06-06 17:47:08,"(6.1066, 52.505084)","(6.106607, 52.505129)",0.005036,0 days 00:00:13
...,...,...,...,...,...,...
13,2023-04-03 17:14:40,2023-04-03 18:47:38,"(5.217039, 53.357044)","(5.973632, 52.203424)",152.892231,0 days 01:32:58
318,2023-06-13 09:29:39,2023-06-13 11:13:35,"(6.002124, 51.185532)","(4.84507, 51.932242)",152.922515,0 days 01:43:56
166,2023-05-10 15:45:33,2023-05-10 17:39:31,"(4.610482, 51.561118)","(5.973216, 52.203351)",167.383830,0 days 01:53:58
268,2023-05-31 15:01:57,2023-05-31 16:46:34,"(4.436424, 52.214414)","(5.973557, 52.203344)",170.926023,0 days 01:44:37
